In [1]:
import json
import warnings
from arctic import Arctic
from pymongo import MongoClient
import pandas as pd

import alpha

warnings.filterwarnings("ignore")

C:\Users\albeadam\Miniconda3\envs\ml4t\lib\site-packages\arctic\_util.py:6: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
C:\Users\albeadam\Miniconda3\envs\ml4t\lib\site-packages\arctic\store\_pandas_ndarray_store.py:6: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import DataFrame, Series, Panel


In [42]:
class Mongo:
    def __init__(self):
        self.client = MongoClient('localhost')
        self.db = self.client.admin

    def server_status(self):
        serverStatusResult = self.db.command("serverStatus")
        return serverStatusResult

    def get_databases(self):
        return self.client.list_database_names()

    def get_collections(self):
        d = dict((db, [collection for collection in self.client[db].list_collection_names()])
                 for db in self.client.list_database_names())
        print(json.dumps(d))
        
    def drop_database(self, name):
        self.client.drop_database(name)
        
class DataStore:
    def __init__(self, name='NASDAQ'):
        mg = Mongo()
        self.store = Arctic('localhost')
        try:
            self.library = self.store[name]
        except:
            self.create_library(name)
            self.library = self.store[name]
        
    def create_library(self, library_name=None):
        if library_name is None:
            print('supply a library name')
            return
        self.store.initialize_library(library_name)
        
    def write(self, name=None, df=None, metadata=None):
        if name is None or df is None:
            print('supply name and/or dataframe')
            return
        self.library.write(name, df)
        
    def delete(self, name=None):
        # function not working - delete is not deleting library
        if name is None:
            print('supply a name to delete')
        self.library.delete(name)
        print(f'{name} deleted')
        
    def list(self):
        return self.store.list_libraries()
    
    def read(self, symbol=None):
        if symbol is None:
            print('supply a library symbol')
            return
        item = self.library.read(symbol)
        return item.data, item.metadata
    
    def append(self, name=None, df=None):
        if name is None or df is None:
            print('supply name or df')
            return
        self.library.append(name, df, upsert=True)

In [38]:
data = DataStore()

{"admin": ["system.version"], "arctic": ["NASDAQ.ARCTIC", "NASDAQ.snapshots", "NASDAQ.version_nums", "NASDAQ", "NASDAQ.versions"], "config": ["system.sessions"], "local": ["startup_log"], "meta_db": ["cache"]}


In [41]:
# d, md = data.read('AAPL_SMA')
# md

In [24]:

toc = alpha.get_toc()
SMA = alpha.technical_indicator(toc[1].replace(' High Usage', ''))

md = SMA['Meta Data']
SMA = SMA['Technical Analysis: SMA']


VersionedItem(symbol=AAPL_SMA,library=arctic.NASDAQ,data=<class 'NoneType'>,version=2,metadata={'1: Symbol': 'AAPL', '2: Indicator': 'Simple Moving Average (SMA)', '3: Last Refreshed': '2021-06-14 16:00:01', '4: Interval': 'weekly', '5: Time Period': 10, '6: Series Type': 'open', '7: Time Zone': 'US/Eastern'},host=localhost)